# Linear regression
    
$$
\mathbf{y} = \mathbf{X}{\boldsymbol \theta} + \epsilon
$$

where:
- $\mathbf{y}$ is a vector of $n$ observed values $y_i\ (i=1,\ldots,n)$.
- $\mathbf{X}$ is a matrix of row-vectors $x_i$ of $n$-dimensional column-vectors.
- ${\boldsymbol \theta} = (\theta_0, \theta_1, \ldots, \theta_p)^{\top}$ is a ($p+1$)-dimensional trainable parameter vector where $\theta_0$ is the bias (intercept).
- $\epsilon$ is the residual.

**References**:

- https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/linear_regression/main.py
- https://en.wikipedia.org/wiki/Linear_regression

### Setup

#### Load packages / modules

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim

#### Generate training dataset

In [1]:
n_examples = 50
slope = 2.0
noise_std = 0.2

In [ ]:
x_train = np.random.rand(n_examples,1)
noise = np.random.randn(n_examples,1)*noise_std
y_train = x_train*slope + noise

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'k.', label='observations')
ax.plot(x_train, x_train*slope, 'tab:gray', label='analytical')
ax.set(xlabel='x', ylabel='y',
       title='train dataset');
ax.legend()

### Setup model

#### Hyperparameters

In [ ]:
input_size = x_train.shape
output_size = y_train.shape

n_epochs = 100
learning_rate = 0.001

#### Model, Loss and Optimizer

In [ ]:
# Linear regression model
model = nn.Linear(input_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss() # Mean-Square Error loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic gradient-descent

### Train the model

In [ ]:
loss_history = []
for epoch in range(n_epochs):
    # Convert numpy arrays to torch tensors
    x = torch.from_numpy(x_train)
    y = torch.from_numpy(y_train)

    # Forward pass
    outputs = model(inputs) # prediction step
    loss = criterion(outputs, targets)
    
    # Backward and optimize
    optimizer.zero_grad() # reset gradient
    loss.backward() # backward propogation 
    optimizer.step() # update gradient
    
    loss_history.append(loss.item())
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss_history, '.-')
ax.set(xlabel='epoch', ylabel='MSE loss',
       title='Loss history');
ax.legend()

### Predict and evaluate

In [ ]:
# Plot the graph
predict = model(torch.from_numpy(x_train)).detach().numpy()

fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'k.', label='observations')
ax.plot(x_train, x_train*slope, 'tab:gray', label='analytical')
ax.plot(x_train, predict, 'tab:red', label='predicted')
ax.set(xlabel='x', ylabel='y',
       title='train dataset');
ax.legend()

### Save model

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')